In [1]:
import cv2, sys
import socket
import threading
import time
import datetime

tello_address = ('192.168.10.1', 8889)
local_address = ('', 9000)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(local_address)

def send(message, delay=0):
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))
    
  time.sleep(delay)

def receive():
  while True:
    try:
      response, ip_address = sock.recvfrom(128)
      print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
      sock.close()
      print("Error receiving: " + str(e))
      break
      
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

send("command", 5)
send("streamon", 5)

capture = cv2.VideoCapture('udp://192.168.10.1:11111') # 텔로 EDU 캠 포트

while True: 
    ret, frame = capture.read()
    if ret == True:
        cv2.imshow('frame', frame)

    key = cv2.waitKey(1);
    if key == ord('q'):
        send("streamoff")
        break
    elif key == ord('s'):
        file = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") +'.jpg'
        cv2.imwrite(file,frame)
        print(file, ' saved')

        
capture.release()
cv2.destroyAllWindows()


Sending message: command
Received message: ok
Sending message: streamon
Received message: ok
Sending message: streamoff
Received message: ok
